In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#declarations
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV

In [ ]:
dir_path = '/content/gdrive/My Drive/Colab/'

df = pd.read_csv(dir_path+"final train.csv") #  processed_1stNov.csv
df.shape

(1482535, 14)

In [ ]:
df.head()

,Unnamed: 0,name,item_condition_id,category_name,brand_name,price,shipping,item_description,log_price,Category,Sub_Category_1,Sub_Category_2,brand_median,combined
0,0,mlb cincinnati reds t shirt size xl,3,Men/Tops/T-shirts,mlb,10.0,1,description yet,2.397895,Men,Tops,T-shirts,2.302585,mlb cincinnati reds t shirt size xl descriptio...
1,1,razer blackwidow chroma keyboard,3,Electronics/Computers & Tablets/Components & P...,razer,52.0,0,keyboard great condition works like came box p...,3.970292,Electronics,Computers & Tablets,Components & Parts,3.713572,razer blackwidow chroma keyboard keyboard grea...
2,2,avaviv blouse,1,Women/Tops & Blouses/Blouse,target,10.0,1,adorable top hint lace key hole back pale pink...,2.397895,Women,Tops & Blouses,Blouse,2.564949,avaviv blouse adorable top hint lace key hole ...
3,3,leather horse statues,1,Home/Home Décor/Home Décor Accents,missing,35.0,1,new tags leather horses retail rm stand foot h...,3.583519,Home,Home Décor,Home Décor Accents,2.639057,leather horse statues new tags leather horses ...
4,4,24k gold plated rose,1,Women/Jewelry/Necklaces,missing,44.0,0,complete certificate authenticity,3.806662,Women,Jewelry,Necklaces,2.639057,24k gold plated rose complete certificate auth...


In [ ]:
y = df['log_price']
df.drop('log_price', axis=1, inplace=True)

In [ ]:
X_train, X_cv, y_train, y_cv = train_test_split(df, y, test_size=0.3, random_state=0)

In [ ]:
del(df)
del(y)

In [ ]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_cv:", X_cv.shape)

print("Shape of Y_train:", y_train.shape)
print("Shape of Y_cv:", y_cv.shape)

Shape of X_train: (1037774, 13)
Shape of X_cv: (444761, 13)
Shape of Y_train: (1037774,)
Shape of Y_cv: (444761,)


WORD TO VEC

Word 2 vec - using Glove vectors

In [ ]:
#please use below code to load glove vectors 
import pickle
with open(dir_path+'glove_vectors', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())

In [ ]:
from tqdm import tqdm

def w2v_vec(preprocessed_essays):
 
  w2v_vectors = []; # the avg-w2v for each sentence/review is stored in this list
 
  for sentence in tqdm(preprocessed_essays): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of 300 length
    word_count = 0 #num of words getting converted into w2v
    #tf_idf_weight =0; # num of words with a valid vector in the sentence/review
 
    for word in sentence.split(): # for each word in a review/sentence
      if (word in glove_words):
        vec = model[word] # getting the vector for each word
        vector += vec # calculating tfidf weighted w2v
        word_count += 1

    #if word_count == 0:
     # print(sentence)
      #raise Exception("Sorry, no numbers below zero")
    if word_count != 0:
      vector = vector/word_count   #len(sentence.split())
 
    w2v_vectors.append(vector)
 
  return w2v_vectors

In [ ]:
X_train_w2v = w2v_vec(X_train['combined'])
X_cv_w2v = w2v_vec(X_cv['combined'])

100%|██████████| 444761/444761 [00:28<00:00, 15854.37it/s]


In [ ]:
print(len(X_train_w2v), len(X_train_w2v[0]))
print(len(X_cv_w2v), len(X_cv_w2v[0]))

1037774 300
444761 300


In [ ]:
# brand_name_processed
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1) #creating an instance
encoder.fit(X_train.brand_name.values.reshape(-1,1))# fitting on train data 

X_train_brand =encoder.transform(X_train.brand_name.values.reshape(-1,1)) #transformin the train data
X_cv_vec_brand = encoder.transform(X_cv.brand_name.values.reshape(-1,1)) # transforming the validation data

print(X_train_brand.shape)
print(X_cv_vec_brand.shape)

(1037774, 1)
(444761, 1)


In [ ]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)#creating an instance
encoder.fit(X_train.Category.values.reshape(-1,1))# fitting on train data 

X_train_category = encoder.transform(X_train.Category.values.reshape(-1,1))
X_cv_category = encoder.transform(X_cv.Category.values.reshape(-1,1))

print(X_train_category.shape)
print(X_cv_category.shape)

(1037774, 1)
(444761, 1)


In [ ]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)#creating an instance
encoder.fit(X_train.Sub_Category_1.values.reshape(-1,1))# fitting on train data 

X_train_Sub_Category_1 = encoder.transform(X_train.Sub_Category_1.values.reshape(-1,1))
X_cv_Sub_Category_1 = encoder.transform(X_cv.Sub_Category_1.values.reshape(-1,1))

print(X_train_Sub_Category_1.shape)
print(X_cv_Sub_Category_1.shape)

(1037774, 1)
(444761, 1)


In [ ]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)#creating an instance
encoder.fit(X_train.Sub_Category_2.values.reshape(-1,1))# fitting on train data 

X_train_Sub_Category_2 = encoder.transform(X_train.Sub_Category_2.values.reshape(-1,1))
X_cv_Sub_Category_2 = encoder.transform(X_cv.Sub_Category_2.values.reshape(-1,1))

print(X_train_Sub_Category_2.shape)
print(X_cv_Sub_Category_2.shape)

(1037774, 1)
(444761, 1)


In [ ]:
x_train_w2 = np.hstack((X_train.item_condition_id.values.reshape(-1,1) , X_train.shipping.values.reshape(-1,1) , 
            X_train_brand , X_train_category , X_train_Sub_Category_1, X_train_Sub_Category_2 , X_train_w2v))

x_train_w2.shape

(1037774, 306)

In [ ]:
x_cv_w2v = np.hstack((X_cv.item_condition_id.values.reshape(-1,1) , X_cv.shipping.values.reshape(-1,1) , 
            X_cv_vec_brand , X_cv_category , X_cv_Sub_Category_1, X_cv_Sub_Category_2 , X_cv_w2v))

x_cv_w2v.shape

(444761, 306)

Standardize the data

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(x_train_w2)
x_train_w2 = scaler.transform(x_train_w2)
x_cv_w2v = scaler.transform(x_cv_w2v)

In [ ]:
#deleting variables to make space in RAM
del(X_train_w2v)
del(X_cv_w2v)
del(X_train)
del(X_cv)
del(X_train_brand)
del(X_train_category)
del(X_train_Sub_Category_1)
del(X_train_Sub_Category_2)

del(model)
del(glove_words)
#del(w2v_vectors)
del(encoder)

del(scaler)

Lasso - Hyper Parameter tuning

In [ ]:
classifier = Lasso()
hyperparams = [{'alpha': [10**x for x in range(-5,3)]}]

grid_cv = GridSearchCV(classifier, hyperparams, cv=None, scoring='neg_root_mean_squared_error', return_train_score=True)

grid_cv.fit(x_train_w2, y_train)

GridSearchCV(estimator=Lasso(),
             param_grid=[{'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10,
                                    100]}],
             return_train_score=True, scoring='neg_root_mean_squared_error')

In [ ]:
print(grid_cv.best_params_)
print(grid_cv.best_score_)

{'alpha': 1e-05}
-0.6589144453702316


In [ ]:
pd.DataFrame(grid_cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,417.062636,9.156045,0.191194,0.038218,0.00001,{'alpha': 1e-05},-0.657267,-0.661059,-0.657066,-0.659540,...,-0.658914,0.001526,1,-0.659080,-0.658139,-0.659134,-0.658518,-0.658489,-0.658672,0.000380
1,285.135636,7.771291,0.165916,0.012856,0.0001,{'alpha': 0.0001},-0.657990,-0.661704,-0.657717,-0.660221,...,-0.659587,0.001513,2,-0.659787,-0.658835,-0.659834,-0.659214,-0.659202,-0.659375,0.000381
2,49.727679,3.559573,0.126351,0.006368,0.001,{'alpha': 0.001},-0.670046,-0.673535,-0.669789,-0.672035,...,-0.671514,0.001408,3,-0.671824,-0.670884,-0.671912,-0.671291,-0.671228,-0.671428,0.000386
3,11.879067,0.072149,0.112427,0.002178,0.01,{'alpha': 0.01},-0.717912,-0.720748,-0.718488,-0.719973,...,-0.719456,0.001073,4,-0.719815,-0.719048,-0.719744,-0.719342,-0.719306,-0.719451,0.000287
4,10.956657,0.027172,0.111225,0.003746,0.1,{'alpha': 0.1},-0.742641,-0.745069,-0.743186,-0.744766,...,-0.744082,0.000976,5,-0.744441,-0.743859,-0.744285,-0.743893,-0.743922,-0.744080,0.000237
5,10.180621,0.020677,0.117208,0.006557,1,{'alpha': 1},-0.748260,-0.750826,-0.748651,-0.750256,...,-0.749669,0.001017,6,-0.750021,-0.749380,-0.749923,-0.749523,-0.749498,-0.749669,0.000254
6,10.144253,0.057789,0.114802,0.001953,10,{'alpha': 10},-0.748312,-0.750899,-0.748709,-0.750330,...,-0.749730,0.001022,7,-0.750081,-0.749440,-0.749984,-0.749583,-0.749559,-0.749729,0.000254
7,9.851693,0.028938,0.109779,0.003487,100,{'alpha': 100},-0.748555,-0.751159,-0.748957,-0.750592,...,-0.749980,0.001027,8,-0.750336,-0.749685,-0.750235,-0.749827,-0.749816,-0.749980,0.000257


From hyperparameter tuning we got the best value for Alpha = 10**-5

In [ ]:
laso = Lasso(alpha = 10**-5)
laso.fit(x_train_w2,y_train)

Lasso(alpha=1e-05)

In [ ]:
# TRAIN PREDICTION
train_pred_l1 = laso.predict(x_train_w2)
# VALIDATION PREDICTION
val_pred_l1 = laso.predict(x_cv_w2v)

In [ ]:
# TRAIN ERROR - without stemming
np.sqrt(mean_squared_error(y_train, train_pred_l1))

0.6586969033112064

In [ ]:
# VALIDATION ERROR -- without stemming
np.sqrt(mean_squared_error(y_cv , val_pred_l1))

0.6566150600647892

Ridge

GridSearchCV to find best Alpha for Ridge

In [ ]:
classifier = Ridge()
hyperparams = [{'alpha': [10**x for x in range(-3,6)]}]

grid_cv = GridSearchCV(classifier, hyperparams, cv=None, scoring='neg_root_mean_squared_error', return_train_score=True)

grid_cv.fit(x_train_w2, y_train)

GridSearchCV(estimator=Ridge(),
             param_grid=[{'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000,
                                    100000]}],
             return_train_score=True, scoring='neg_root_mean_squared_error')

In [ ]:
pd.DataFrame(grid_cv.cv_results_, columns = ['params', 'mean_test_score', 'std_test_score','rank_test_score', 'mean_train_score'])

,params,mean_test_score,std_test_score,rank_test_score,mean_train_score
0,{'alpha': 0.001},-0.658906,0.001529,5,-0.658660
1,{'alpha': 0.01},-0.658906,0.001529,4,-0.658660
2,{'alpha': 0.1},-0.658906,0.001529,3,-0.658660
3,{'alpha': 1},-0.658906,0.001529,2,-0.658660
4,{'alpha': 10},-0.658906,0.001527,1,-0.658661
5,{'alpha': 100},-0.658946,0.001518,6,-0.658708
6,{'alpha': 1000},-0.660301,0.001485,7,-0.660107
7,{'alpha': 10000},-0.670739,0.001408,8,-0.670648
8,{'alpha': 100000},-0.699501,0.001155,9,-0.699477


We are getting best results for alpha = 1 or 10. Using this value to train the regressor again

In [ ]:
ridge_best = Ridge(alpha=1)
ridge_best.fit(x_train_w2, y_train)

Ridge(alpha=1)

In [ ]:
# TRAIN PREDICTION
train_pred_l2 = ridge_best.predict(x_train_w2)
# VALIDATION PREDICTION
val_pred_l2 = ridge_best.predict(x_cv_w2v)

In [ ]:
print(np.sqrt(mean_squared_error(y_train, train_pred_l2)))
print(np.sqrt(mean_squared_error(y_cv , val_pred_l2)))

0.6586848126857683
0.6566140528402704


Decision Tree

Manual search for hyper-parameter tuning

In [ ]:
from sklearn.tree import DecisionTreeRegressor

def findbestparameter(depth, x_tr, y_tr, x_cv, y_cv):

  dt_best = DecisionTreeRegressor(criterion="squared_error",max_depth=depth, max_features="auto")
  dt_best.fit(x_tr, y_tr)
  predict_cv = dt_best.predict(x_cv)
  print("CV error for Max_depth:{} is".format(depth), np.sqrt(mean_squared_error(y_cv , predict_cv)))

for depth in [7,10,12,15]: 
  findbestparameter(depth, x_train_w2, y_train, x_cv_w2v, y_cv)

CV error for Max_depth:7 is 0.6638750300656487
CV error for Max_depth:10 is 0.6386888791521571
CV error for Max_depth:12 is 0.6289284779131978
CV error for Max_depth:15 is 0.6455573021802852


In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt_best = DecisionTreeRegressor(criterion="squared_error",max_depth=12, max_features="auto")
dt_best.fit(x_train_w2, y_train)


DecisionTreeRegressor(max_depth=12, max_features='auto')

In [ ]:
# TRAIN PREDICTIONS
train_pred_dt = dt_best.predict(x_train_w2)
# VALIDATION PREDICTIONS
val_pred_dt = dt_best.predict(x_cv_w2v)

In [ ]:
# TRAIN ERROR
np.sqrt(mean_squared_error(y_train , train_pred_dt))

0.6311514825223409

In [ ]:
# VALIDATION ERROR
np.sqrt(mean_squared_error(y_cv , val_pred_dt))

0.6386873446816325

Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
  
# create regressor object
rf_regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)

rf_regressor.fit(x_train_w2, y_train)

RandomForestRegressor(n_estimators=10, random_state=0)

took 1-2 hours for n_estimators = 10, hence not yet ran for more estimators.

In [ ]:
# TRAIN PREDICTION
train_pred_rf = rf_regressor.predict(x_train_w2)
# VALIDATION PREDICTION
val_pred_rf = rf_regressor.predict(x_cv_w2v)

In [ ]:
print(np.sqrt(mean_squared_error(y_train, train_pred_rf)))
print(np.sqrt(mean_squared_error(y_cv , val_pred_rf)))

0.25134413592257443
0.582917633372024


Light Gradient Boosting Machine

In [ ]:
#pip install lightgbm

In [ ]:
import lightgbm as lgb
from lightgbm import LGBMRegressor

In [ ]:
def find_best_lgb_parameter(estimator, x_tr, y_tr, x_cv, y_cv):

  model = LGBMRegressor(n_estimators = estimator)
  model.fit(x_tr, y_tr)
  predict_cv = model.predict(x_cv)
  print("CV error for Max_depth:{} is".format(estimator), np.sqrt(mean_squared_error(y_cv , predict_cv)))

for estimator in [10,100,1000,1200,1300,1400]: 
  find_best_lgb_parameter(estimator, x_train_w2, y_train, x_cv_w2v, y_cv)

CV error for Max_depth:10 is 0.6723209364480769
CV error for Max_depth:100 is 0.5787502133352906
CV error for Max_depth:1000 is 0.518565024675861
CV error for Max_depth:1200 is 0.5159646306057264
CV error for Max_depth:1300 is 0.5147603388807999
CV error for Max_depth:1400 is 0.513684187020763


In [ ]:
model = LGBMRegressor(n_estimators = 1700)
model.fit(x_train_w2, y_train)

predict_tr = model.predict(x_train_w2)
predict_cv = model.predict(x_cv_w2v)

In [ ]:
#for n_estimators = 1500
print(np.sqrt(mean_squared_error(y_train, predict_tr)))
print(np.sqrt(mean_squared_error(y_cv , predict_cv)))

0.4818167984917683
0.5128741548642093


In [ ]:
#for n_estimators = 1600
print(np.sqrt(mean_squared_error(y_train, predict_tr)))
print(np.sqrt(mean_squared_error(y_cv , predict_cv)))

0.47888745237746705
0.5119911827873267


In [ ]:
#for n_estimators = 1700
print(np.sqrt(mean_squared_error(y_train, predict_tr)))
print(np.sqrt(mean_squared_error(y_cv , predict_cv)))

0.4758793211089417
0.5110757335586951


Collating all results

In [1]:
from prettytable import PrettyTable

table = PrettyTable()
table.field_names = ["Model", "CV Error"]

table.add_row(['Benchmark','0.7438'])
table.add_row(['Lasso','0.6566'])
table.add_row(['Ridge','0.6566'])
table.add_row(['Decision Tree','0.6311'])
table.add_row(['Random Forest','0.5829'])
table.add_row(['LGBM','0.5110'])

print(table)

+---------------+----------+
|     Model     | CV Error |
+---------------+----------+
|   Benchmark   |  0.7438  |
|     Lasso     |  0.6566  |
|     Ridge     |  0.6566  |
| Decision Tree |  0.6311  |
| Random Forest |  0.5829  |
|      LGBM     |  0.5110  |
+---------------+----------+
